In [50]:
!pip install transformers -U -q
import torch
import tensorflow as tf

In [51]:
! pip install sentencepiece

In [52]:
!pip freeze | grep transformers

transformers==4.34.0


In [53]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [54]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")

In [55]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")


In [280]:

# input sentences
input_text = ["Ram and Shyam are fighting"]

# convert sentences to tensors
model_inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

In [281]:
# translate from English to Hindi
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
)

translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

print(translation)

['राम और श्याम लड़ रहे हैं']


In [282]:
import nltk
nltk.download('averaged_perceptron_tagger')

def is_noun(text):
    ans = nltk.pos_tag([text])
    val = ans[0][1]

    if val in ('NN', 'NNS', 'NNPS', 'NNP'):
        return True
    else:
        return False

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\anupa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [283]:
# Split the input text into words
words = input_text[0].split()

# Initialize a list to store nouns
noun_list = []

# Check if each word is a noun and add it to the list
for word in words:
    if is_noun(word):
        noun_list.append(word)

print(noun_list)

['Ram', 'Shyam']


In [284]:
print(type(noun_list[0]))

<class 'str'>


In [285]:
# Create a copy of the translation for replacement
translated_sentence = translation.copy()
translated_sentence = translated_sentence[0]

# Translate English nouns to Hindi and replace them in the translated_sentence
for noun in noun_list:
    noun = noun.replace(',', '')
    model_inputs_noun = tokenizer(noun, return_tensors="pt", padding=True, truncation=True)

    generated_tokens_noun = model.generate(
        **model_inputs_noun,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
    )

    noun_translation = tokenizer.batch_decode(generated_tokens_noun, skip_special_tokens=True)[0]
#     print(noun_translation)
    # Replace the noun_translation (a list) with the original noun (a string)
    translated_sentence = translated_sentence.replace(noun_translation ,noun)

print(translated_sentence)

Ram और Shyam लड़ रहे हैं
